<a href="https://colab.research.google.com/github/rahul2348/WINTER-PROJECT-ACTIVE-LEARNING/blob/main/asiigment_3_230828.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torchvision import models

model = models.resnet18(pretrained=True)


In [ ]:
def least_confidence(predictions):
    return 1 - predictions.max(dim=1).values


In [ ]:
import torch

def prediction_entropy(predictions):
    return -torch.sum(predictions * torch.log(predictions + 1e-9), dim=1)


In [ ]:
def margin_sampling(predictions):
    top_two = predictions.topk(2, dim=1).values
    return top_two[:, 0] - top_two[:, 1]


In [ ]:
from torch.nn.functional import cosine_similarity

def minimize_cosine_similarity(features):
    similarity_matrix = cosine_similarity(features.unsqueeze(0), features.unsqueeze(1), dim=2)
    return similarity_matrix


In [ ]:
def l2_norm(features):
    pairwise_distance = torch.cdist(features, features, p=2)
    return pairwise_distance


In [ ]:
from torch.nn.functional import kl_div

def kl_divergence(predictions, reference_distribution):
    return kl_div(predictions.log(), reference_distribution, reduction='batchmean')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64*12*12, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(torch.relu(self.conv2(x)), 2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [ ]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 0.0302
Epoch [2/10], Loss: 0.0209
Epoch [3/10], Loss: 0.6277
Epoch [4/10], Loss: 0.0030
Epoch [5/10], Loss: 0.0004
Epoch [6/10], Loss: 0.0063
Epoch [7/10], Loss: 0.0038
Epoch [8/10], Loss: 0.0000
Epoch [9/10], Loss: 0.0000
Epoch [10/10], Loss: 0.0017
